In [1]:
import csv
import json
import sys
import pandas as pd
import os
import copy
import math
import json
# from bitmap import BitMap
from collections import defaultdict
import numpy as np
import itertools as it
import time

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz    
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip3 install -q findspark
!pip3 install -q pyspark
!pip3 install -q kaggle
!pip3 install bitmap
from bitmap import BitMap

from google.colab import files
print("Upload kaggle.json to use kaggle APIs")
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

# !curl -OL https://raw.githubusercontent.com/Bertonc98/ProgettoAMD/main/sample.csv

!kaggle datasets files "ashirwadsangwan/imdb-dataset"
!kaggle datasets download "ashirwadsangwan/imdb-dataset" -f "title.principals.tsv.gz"
!unzip "title.principals.tsv.gz.zip"
!rm "title.principals.tsv.gz.zip"
!gunzip -k "title.principals.tsv.gz"
!rm "title.principals.tsv.gz"

     |████████████████████████████████| 212.4MB 60kB/s 
     |████████████████████████████████| 204kB 16.0MB/s 
Upload kaggle.json to use kaggle APIs


Saving kaggle.json to kaggle.json
kaggle.json
name                      size  creationDate         
-----------------------  -----  -------------------  
title.ratings.tsv.gz       5MB  2019-11-18 18:36:47  
title.principals.tsv.gz  290MB  2019-11-18 18:36:47  
name.basics.tsv.gz       180MB  2019-11-18 18:36:47  
title.basics.tsv.gz      108MB  2019-11-18 18:36:47  
title.akas.tsv.gz        158MB  2019-11-18 18:36:47  
 94% 270M/287M [00:03<00:00, 79.0MB/s]
100% 287M/287M [00:03<00:00, 82.9MB/s]
Archive:  title.principals.tsv.gz.zip
  inflating: title.principals.tsv.gz  


In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

home = os.environ["SPARK_HOME"]
os.environ["PYTHONPATH"] = home+"/python"+home+"/python/lib/py4j-0.10.9-src.zip:"+os.environ["PYTHONPATH"]

In [4]:
import findspark
from pyspark.sql import SparkSession
import pyspark
from pyspark import SparkConf
findspark.init()
conf = SparkConf()
conf.set("spark.driver.memory", "5g")
conf.set("spark.executor.memory", "5g")
spark = SparkSession.builder.config(conf = conf).master("local[*]").getOrCreate()

In [5]:
sourceFile = 'title.principals.tsv'

data = spark.read.options(delimiter = "\t", header=True).csv(sourceFile)

from pyspark.sql.window import Window

from pyspark.sql.functions import monotonically_increasing_id,row_number

# rdd_result = data.select(['nconst', 'category'])\
#             .where("category == 'actress' or category == 'actor' ")\
#             .select(['nconst'])\
#             .distinct()

# rdd_result = rdd_result.withColumn("idx",row_number().over(Window.orderBy(monotonically_increasing_id())))

#ALTERNATIVA
filter_actors = lambda row: row["category"] in ["actor", "actress"]
map_actors = lambda row: (row["tconst"], int(row["nconst"][2:])) 

rdd_basket = data.rdd.filter(filter_actors)\
                 .map(map_actors)\
                 .groupByKey()\
                 .mapValues(list)\
                 .values()

In [ ]:
# this dictionary maps all the items to an int
# itemToInt = {key:value for (key, value) in rdd_result.collect()}

In [6]:
sample = rdd_basket.sample(False, 0.0001)
sample.cache()

PythonRDD[19] at RDD at PythonRDD.scala:53

In [7]:
sample.count()

361

In [48]:
#Apriori
def get_minimum(chunk):
  mi = 10000000000000
  for basket in chunk:
    m = min(list(map(int, basket)))
    mi = min(m, mi)
  return mi

def get_combinations(basket, size):
    return it.combinations(sorted(basket), size)

def update_frequency(l, item):
    if item + 1 > len(l):
        l.extend((item+1-len(l))*[0])
    l[item] += 1
    
def singleton_pairs_frequency_apriori(chunk, threshold, m):
    singleton_freq = []
    for basket in chunk:
        basket = list(map(int, basket))
        for item in basket:
            update_frequency(singleton_freq, (item-m))
    return filter_singletons(singleton_freq, m, threshold)

# this function filter the given singletons with the absolute freqeuncy, deleting the ones with
# frequency below the threshold
def filter_singletons(singleton, m, threshold):
    print(singleton)
    return [(iid+m,) for iid, freq in enumerate(singleton) if freq>=threshold]    

def get_joins(tuples, k):
    #to obtain an element of lenght k:
    #join 2 elements of length k-1 with the first k-1 elements in common
    l = []
    for i in range(len(tuples)-1):
        for j in range(i+1, len(tuples)):
            if(tuples[i][:k-2] == tuples[j][:k-2]):
                l += [tuples[i] + tuples[j][k-2:]]
    return l

def get_kth_apriori(chunk, precedents, threshold, k):
    kth = {tpl:0 for tpl in get_joins(precedents, k)}

    for basket in chunk:
        # basket = list(map(int, basket))
        tpls = list(get_combinations(basket, k))
        for t in tpls:
          try:
            kth[t] += 1
          except:
            pass
        # for i in range(len(kth)):
        #     if kth[i] in tpls:
        #         counters[i] += 1

    return [i for i in kth.keys() if kth[i] >= threshold]

In [49]:
def apriori_main(partition, threshold):
      generator, chunk = it.tee(partition, 2)
      frequent_sets = []
      #get minimum
      m = get_minimum(chunk)
      chunk = copy.copy(generator)
      singleton = singleton_pairs_frequency_apriori(chunk, threshold, m)
      
      frequent_sets += singleton
      
      k = 2
      result = ["placeholder"]
      while result != []:
          
          precedents = [i for i in frequent_sets if len(i) == k-1 ]
          
          chunk = copy.copy(generator)
          result = get_kth_apriori(chunk, precedents, threshold, k)
          
          frequent_sets = frequent_sets + result
          #print("Found frequent itemset of length: ",k)
          k += 1
      return frequent_sets

def apriori(threshold):
  def main(partition):
      return apriori_main(partition, threshold)
  return main

In [50]:
#Apriori total
# t = math.ceil(sample.count()/ 1000 )
t = 2
start_time = time.time()
result_apriori = apriori_main(sample.toLocalIterator(), t)
print("--- %s seconds ---" % (time.time() - start_time))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



--- 2.782212018966675 seconds ---


In [13]:
#PCY
def hashF(element, size):
    return hash(element)%size

def singleton_pairs_frequency_pcy(chunk, hash_size, threshold, m):
    singleton_freq = []
    hashmap = np.zeros(hash_size)
    for basket in chunk:
        basket = list(map(int, basket))
        for item in basket:
            update_frequency(singleton_freq, item-m)
        for pair in get_combinations(basket, 2):
            hashmap[hashF(pair, hash_size)] += 1
    return filter_singletons(singleton_freq, m, threshold), hashmap 

# this function collapse the hashmap given in input into a bitmap of size "size" and set a 1 in the bitmap if the value in
# the hashmap is over the the given threshold
def hash_to_bitmap(hashmap, size, threshold):
    bmap = BitMap(size)
    for position in range(len(hashmap)):
        if hashmap[position] >= threshold:
            bmap.set(position)
    del hashmap
    return bmap

def get_kth(chunk, precedents, hash_size, bmap, threshold, k):
    kth = {tpl:0 for tpl in get_joins(precedents, k) if bmap[hashF(tpl, hash_size)]}
    next_hashmap = np.zeros(hash_size)
    
    for basket in chunk:
        # basket = list(map(int, basket))
        tpls = list(get_combinations(basket, k))
        for t in tpls:
          try:
            kth[t] += 1
          except:
            pass
        for kth_tuples in get_combinations(basket, k+1):
            next_hashmap[hashF(kth_tuples, hash_size)] += 1

    return [k for k,v in kth.items() if v >= threshold], next_hashmap

In [14]:
def pcy_main(partition, threshold, hash_size):
      generator, chunk = it.tee(partition, 2)
      frequent_sets = []
      m = get_minimum(chunk)
      chunk = copy.copy(generator)
  #     phase 1: compute frequency of singletons and pairs; filter nonfrequent singletons
      singleton, hashmap = singleton_pairs_frequency_pcy(chunk, hash_size, threshold, m)
  #     phase 2: collpse the hahsmap into a bitmap
      bitmap = hash_to_bitmap(hashmap, hash_size, threshold)
  #     phase 3
  #     frequent_sets contains all the sets of all the kth length that are frequent
  #     bitmap is the bitmap mapping the sets of length k+1
  #     frequent_sets, bitmap = frequent_couple(data, singleton, hash_size, bitmap, threshold)
      frequent_sets += singleton
      
      k = 2
      result = ["placeholder"]
      while result != []:
          precedents = [i for i in frequent_sets if len(i) == k-1 ]
          chunk = copy.copy(generator)
          result, hashmap = get_kth(chunk, precedents, hash_size, bitmap, threshold, k)
          bitmap = hash_to_bitmap(hashmap, hash_size, threshold)
          frequent_sets = frequent_sets + result
          # print("Found frequent itemset of length: ",k)
          k += 1
          
      return frequent_sets

def pcy(threshold, hash_size):
# def main(chunk, threshold, hash_size):
  def main(partition):
    return pcy_main(partition, threshold, hash_size)
  return main

In [25]:
#PCY total
# t = math.ceil(sample.count()/500)
t = 2
start_time = time.time()
hash_size = 504
result_pcy = pcy_main(sample.toLocalIterator(), t, hash_size)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.453624963760376 seconds ---


In [39]:
sorted(result_apriori) == sorted(frequent_apriori_son)

False

In [40]:
[x for x in result_apriori if x not in frequent_apriori_son]

[(7429539,)]

PythonRDD[19] at RDD at PythonRDD.scala:53

In [10]:
# SON

# phase 1 scan all baskets 
# -input: id_movie, list(int_actor)
# -compute frequency (count)
# -count combinations (size 2) into hashmap
# -filter frequent singletons (count >= s)
# -output: int_actor: count
def son(candidates):
  def son_main(partition):
    count = {item: 0 for item in candidates}
    for basket in partition:
      for c in candidates:
          if c in get_combinations(basket, len(c)):
            count[c] += 1
    return [(k, v) for k, v in count.items()]
  return son_main

In [36]:
#SON Apriori
p = 1/sample.getNumPartitions()
#to fix
# threshold = math.ceil(sample.count()/1000)
# s_threshold = math.ceil(0.95 * threshold * p)
threshold = 2
s_threshold = 1
start_time = time.time()
candidates = sample.mapPartitions(apriori(s_threshold))\
                   .map(lambda item: (1,[item]))\
                   .reduceByKey(lambda a, b: list(set(a + b )))\
                   .values()\
                   .collect()[0]
print(candidates.count((7429539,)))
frequent_apriori_son = sample.mapPartitions(son(candidates))\
                            .reduceByKey(lambda a,b: a+b)\
                            .filter(lambda item: item[1] >= threshold)\
                            .map(lambda item: item[0])\
                            .collect()
print("--- %s seconds ---" % (time.time() - start_time))

1
--- 36.77401542663574 seconds ---


In [21]:
#SON PCY
p = 1/sample.getNumPartitions()
#to fix
# threshold = math.ceil(sample.count()/500)
threshold = 2
# s_threshold = math.ceil(0.95 * threshold * p)
s_threshold = 1
hash_size = 504
start_time = time.time()
candidates = sample.mapPartitions(pcy(s_threshold, hash_size))\
                   .map(lambda item: (1,[item]))\
                   .reduceByKey(lambda a, b: list(set(a + b )))\
                   .values()\
                   .collect()[0]
frequent_pcy_son = sample.mapPartitions(son(candidates))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda item: item[1] >= threshold)\
                 .map(lambda item: item[0])\
                 .collect()
print("--- %s seconds ---" % (time.time() - start_time))

--- 37.00053119659424 seconds ---


In [24]:
sorted(frequent_pcy_son) == sorted(frequent_apriori_son)

True

In [ ]:
# TOIVONEN
# V Take a sample of the total, a fraction p€[0,1] (adjust the threshold s = p * s)
# V Run Apriori over it and obtain the candidates frequent itemset from the sample
# V Build the negative border from candidates
# Scan all the basket and check for itemset in candidates or in negative border:
#   - If no element from the negative border is frequent return candidates as frequent itemsets
#   - Otherwise no answer, need to rerun with different sample or threshold

# sample.filter(lambda basket: basket not in candidates)

p = 1/sample.getNumPartitions() #to fix
t = math.ceil(sample.count()/1000)
hash_size = 504 #this because we run PCY to find candidates
candidates = sample.coalesce(1)\
                   .mapPartitions(pcy(t, hash_size))\
                   .map(lambda item: (1,[item]))\
                   .reduceByKey(lambda a, b: list(set(a + b )))\
                   .values()\
                   .collect()[0]
# nb = negative_border(candidates)

In [ ]:
# frequent_s = sorted([s[0] for s in candidates if s not in nb and len(s)==1])
# nb += [el for el in list(it.combinations(frequent_s, 2)) if el not in candidates]
def negative_border(candidates, sample):
  diff = sample.flatMap(lambda xs: [x for x in xs])\
              .distinct()\
              .map(lambda el: (el,))\
              .filter(lambda el: el not in candidates)\
              .collect()
  nb = diff
  #singleton frequenti
  prev_freq = [c for c in candidates if len(c) == 1]
  k=2
  while diff!=[]:
    candidates_k = [el for el in candidates if len(el) == k]
    diff = [kth for kth in get_joins(prev_freq, k) if kth not in candidates_k]
    nb += diff
    prev_freq = candidates_k
    k += 1
  return nb

nb = dict.fromkeys(negative_border(candidates, sample), 0)
c_dict = dict.fromkeys(candidates, 0)

In [ ]:
def update(basket):
  l = len(basket)
  b = sorted(basket)
  lista = []
  for i in range(l):
    for el in it.combinations(b, i+1):
      try:
        nb[el] += 1
      except:
        pass
      
      try:
        c_dict[el] += 1
      except:
        pass
  return None

for basket in rdd_basket.toLocalIterator():
  update(basket)
t2 = math.ceil(rdd_basket.count()/1000)
buleano = True
for v in nb.values():
  if (v > t2):
    buleano = False
    break
buleano
if (buleano):
  c_dict = {k:v for (k:v) in c_dict if v>t2}


In [ ]:
# f = csv.reader(open("intToItem.csv", encoding="utf-8", newline="\n"), delimiter=";")
# intToItem = { int(k):v for (k, v) in f}

In [ ]:
# def getItemIdFromInt(l):
#   return [intToItem[k] for k in l[0]]
def getItemIdFromInt(l):
  return ["nm" + str(k).zfill(7) if len(str(k))<7 else  "nm" + str(k) for k in l]

frequentId = frequent.map(getItemIdFromInt)

In [ ]:
!kaggle datasets download "ashirwadsangwan/imdb-dataset" -f "name.basics.tsv.gz"
!unzip "name.basics.tsv.gz.zip"
!rm "name.basics.tsv.gz.zip"
!gunzip -k "name.basics.tsv.gz"
!rm "name.basics.tsv.gz"

 93% 167M/180M [00:01<00:00, 114MB/s]
100% 180M/180M [00:01<00:00, 104MB/s]
Archive:  name.basics.tsv.gz.zip
  inflating: name.basics.tsv.gz      


In [ ]:
nameTable = spark.read.options(delimiter = "\t", header=True).csv("name.basics.tsv")
# idName = {id:name for (id, name) in nameTable.select(["nconst","primaryName"]).collect()}

In [ ]:
flattened_frequentId = frequentId.flatMap(lambda xs: [x for x in xs]).collect()

nameTable = nameTable.filter(nameTable.nconst.isin(flattened_frequentId))\
                     .select(["nconst", "primaryName"])\
                     .collect()
nameTable = {id : name for (id, name) in nameTable}

In [ ]:
frequentId = frequentId.collect()
basket_translated = []
for basket in frequentId:
  basket_translated.append([nameTable[id] for id in basket])
basket_translated